In [2]:
!pip install datasets transformers wandb sentencepiece tqdm

     |████████████████████████████████| 325 kB 17.4 MB/s 
     |████████████████████████████████| 4.0 MB 58.4 MB/s 
     |████████████████████████████████| 1.8 MB 53.7 MB/s 
     |████████████████████████████████| 1.2 MB 52.0 MB/s 
     |████████████████████████████████| 212 kB 61.9 MB/s 
     |████████████████████████████████| 1.1 MB 49.8 MB/s 
     |████████████████████████████████| 136 kB 64.7 MB/s 
     |████████████████████████████████| 77 kB 8.2 MB/s 
     |████████████████████████████████| 127 kB 65.5 MB/s 
     |████████████████████████████████| 6.6 MB 42.6 MB/s 
     |████████████████████████████████| 895 kB 69.7 MB/s 
     |████████████████████████████████| 596 kB 65.8 MB/s 
     |████████████████████████████████| 181 kB 60.6 MB/s 
     |████████████████████████████████| 144 kB 61.5 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 94 kB 4.9 MB/s 
     |████████████████████████████████| 271 kB 82.1 MB/s 
     |██████████████

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
from datasets import load_dataset
from functools import partial

import random
import logging
from tqdm.auto import tqdm
from torch.utils.data import DataLoader


import wandb
import transformers

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small', force_download=True)

Downloading:   0%|          | 0.00/687k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [6]:
dataset = load_dataset('spider')

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Dataset spider downloaded and prepared to /root/.cache/huggingface/datasets/spider/spider/1.0.0/79778ebea87c59b19411f1eb3eda317e9dd5f7788a556d837ef25c3ae6e5e8b7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def preprocess_function(examples, tokenizer, max_seq_length):
    

    inputs = examples['question']
    targets = examples['query']
    
    model_inputs = tokenizer(inputs, max_length=max_seq_length, padding="max_length", truncation=True)
    target_ids = tokenizer(targets, max_length=max_seq_length, padding="max_length", truncation=True)
    target_ids = target_ids.input_ids
    
    #decoder_input_ids = []

    # for target in target_ids:
    #     decoder_input_ids.append([tokenizer.bos_token_id] + target)
    #     labels.append(target + [tokenizer.eos_token_id])

    # model_inputs["decoder_input_ids"] = decoder_input_ids

    labels_with_ignore_index = []
    
    for labels_example in target_ids:
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)
    
    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs


In [18]:
max_seq_length=128
overwrite_cache=True
preprocessing_num_workers = 8
batch_size=32
num_train_epochs=5
device='cpu'
learning_rate=1e-5
weight_decay=0.01
lr_scheduler_type = 'linear'
num_warmup_steps = 0
max_train_steps = 20000
logging_steps=25
eval_every_step=50

In [9]:
dataset["train"].column_names

['db_id',
 'query',
 'question',
 'query_toks',
 'query_toks_no_value',
 'question_toks']

In [10]:
column_names = [c for c in dataset["train"].column_names if c not in ['query', 'db_id']]

preprocess_function_wrapped = partial(
    preprocess_function,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
)


processed_datasets = dataset.map(
    preprocess_function_wrapped,
    batched=True,
    num_proc=preprocessing_num_workers,
    remove_columns=column_names,
    load_from_cache_file=not overwrite_cache,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Running tokenizer on dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #4:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #5:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #6:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #7:   0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
processed_datasets.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"] if "validation" in processed_datasets else processed_datasets["test"]

# Log a few random samples from the training set:
for index in random.sample(range(len(train_dataset)), 2):
    print(f"Sample {index} of the training set: {train_dataset[index]}.")
    print(f"Decoded input_ids: {tokenizer.decode(train_dataset[index]['input_ids'])}")
    print(f"Decoded labels: {tokenizer.decode([label for label in train_dataset[index]['labels'] if label != -100])}")
    print("\n")


Sample 6839 of the training set: {'input_ids': tensor([    1,  3125,   326,  1300,   434,  3775, 23350,  4363,  1492,   854,
          326, 20456,   434,   326,  3986,  3036,   304,   432,   481,  3548,
           18,     2,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,   

In [12]:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, batch_size=batch_size
)

eval_dataloader = DataLoader(
    eval_dataset, shuffle=False, batch_size=batch_size
)

In [13]:
type(train_dataset['attention_mask'][0])

torch.Tensor

In [14]:
next(iter(train_dataloader))['attention_mask']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [19]:
import torch.nn as nn

class CodeT5_NLSQL(nn.Module):

  def __init__(self, model):
    super().__init__()

    self.model = model
    #self.input_size = model.config.to_dict()['hidden_size']
    #self.num_classes = num_classes

    #self.input_layer = nn.Linear(self.input_size)

  def forward(self, input_ids, attention_mask, labels=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    return outputs

In [16]:
import torch

In [20]:
nlsql_model = CodeT5_NLSQL(model)
nlsql_model.to(device)

CodeT5_NLSQL(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32100, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32100, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=512, bias=False)
                (k): Linear(in_features=512, out_features=512, bias=False)
                (v): Linear(in_features=512, out_features=512, bias=False)
                (o): Linear(in_features=512, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseReluDense(
                (wi): Linear(in_features=512, out_features=2048, bias=False)
                (wo): Linear(in_featur

In [21]:
import tqdm

In [22]:
dataset['validation']

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 1034
})

In [23]:
from tqdm.auto import tqdm

gold_file = open("gold.txt", "w")

gold_queries = []

for row in dataset['validation']:
    gold_queries.append(row['query'] + '\t' + row['db_id'])
    
gold_file.write("\n".join(gold_queries))
  
gold_file.close()

In [27]:
import numpy as np 
from evaluation import evaluate, build_foreign_key_map_from_json

def evaluate_model(model, dataloader, tokenizer, max_seq_length, device):
  model.eval()

  all_preds = []
  all_labels = []

  avg_batch_acc = 0
  pred_file = open("pred.txt", "w")

  with torch.no_grad():
    for batch in tqdm(dataloader, desc="Evaluation"):
      input_ids = batch["input_ids"].to(device)
      labels = batch["labels"].to(device)

      attention_mask = batch["attention_mask"].to(device)
      #token_type_ids = batch["token_type_ids"].to(device)
      
      generated_tokens = model.model.generate(
            input_ids,
            max_length=max_seq_length,
            # beam_size=beam_size,
        )


      #logits = model(input_ids=input_ids, labels=labels, attention_mask=attention_mask)

      #preds = torch.argmax(logits, dim=-1)
      #metric.add_batch(predictions=preds, references=labels)
      decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
      
      # for row in decoded_preds:
      #   all_preds.append(row)

      labels = labels.tolist()

  
      for row in decoded_preds:
          all_preds.append("".join(row))
      

      new_labels = []
      for label_row in labels:
        if ' ' in label_row:
          print("we found a space")
        new_labels.append([value for value in label_row if value != -100])
        
      decoded_labels = tokenizer.batch_decode(new_labels, skip_special_tokens=True)


  pred_file.write("\n".join(all_preds))


  pred_file.close()
  scores = evaluate('gold.txt', 'pred.txt', '/content/drive/MyDrive/database/', 'match', build_foreign_key_map_from_json('tables.json'))
  
  evaluation_results = {
      "eval/exact_match": scores['all']['exact']
  }



  model.train()
  return evaluation_results, input_ids, decoded_preds, decoded_labels


In [25]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)


lr_scheduler = transformers.get_scheduler(
    name=lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=max_train_steps,
)

In [26]:
!ls "/content/drive/MyDrive/database"

academic			 journal_committee
activity_1			 loan_1
aircraft			 local_govt_and_lot
allergy_1			 local_govt_in_alabama
apartment_rentals		 local_govt_mdm
architecture			 machine_repair
assets_maintenance		 manufactory_1
baseball_1			 manufacturer
battle_death			 match_season
behavior_monitoring		 medicine_enzyme_interaction
bike_1				 mountain_photos
body_builder			 movie_1
book_2				 museum_visit
browser_web			 music_1
candidate_poll			 music_2
car_1				 music_4
chinook_1			 musical
cinema				 network_1
city_record			 network_2
climbing			 news_report
club_1				 orchestra
coffee_shop			 party_host
college_1			 party_people
college_2			 performance_attendance
college_3			 perpetrator
company_1			 pets_1
company_employee		 phone_1
company_office			 phone_market
concert_singer			 pilot_record
county_public_safety		 poker_player
course_teach			 product_catalog
cre_Doc_Control_Systems		 products_for_hire
cre_Docs_and_Epenses		 products_gen_characteristics
cre_Doc_Template_Mgt		 program_share

In [ ]:
run = wandb.init(project=f"CODET5_SQLNL")

global_step = 0

progress_bar = tqdm(range(len(train_dataloader) * num_train_epochs))


# iterate over epochs
for epoch in range(num_train_epochs):
    nlsql_model.train()  # make sure that model is in training mode, e.g. dropout is enabled

    # iterate over batches
    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        # decoder_input_ids = batch["decoder_input_ids"].to(device)
        # key_padding_mask = batch["encoder_padding_mask"].to(device)
        labels = batch["labels"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = nlsql_model(
            input_ids=input_ids,
            labels=labels,
            attention_mask=attention_mask
        )

        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)
        global_step += 1

        wandb.log(
            {
                "train_loss": loss,
                "learning_rate": optimizer.param_groups[0]["lr"],
                "epoch": epoch,
            },
            step=global_step,
        )


        if global_step % eval_every_step == 0:
            eval_results, last_input_ids, last_decoded_preds, last_decoded_labels = evaluate_model(
                model=nlsql_model,
                dataloader=eval_dataloader,
                tokenizer=tokenizer,
                device=device,
                max_seq_length=max_seq_length,

            )    
            wandb.log(
              {"eval/exact_match": eval_results['eval/exact_match']},
              step=global_step,
            )
            print("Generation example:")
            random_index = random.randint(0, len(last_input_ids) - 1)
            print(f"Input sentence: {tokenizer.decode(last_input_ids[random_index], skip_special_tokens=True)}")
            print(f"Generated sentence: {last_decoded_preds[random_index]}")
            print(f"Reference sentence: {last_decoded_labels[random_index]}")


        if global_step % logging_steps == 0:
            # An extra training metric that might be useful for understanding
            # how well the model is doing on the training set.
            # Please pay attention to it during training.
            # If the metric is significantly below 80%, there is a chance of a bug somewhere.
            predictions = logits.argmax(-1)

            label_nonpad_mask = labels != tokenizer.pad_token_id
            num_words_in_batch = label_nonpad_mask.sum().item()

            accuracy = (predictions == labels).masked_select(label_nonpad_mask).sum().item() / num_words_in_batch

            wandb.log(
                {"train_batch_word_accuracy": accuracy},
                step=global_step,
            )
            
            # print("Generation example:")
            # random_index = random.randint(0, len(last_input_ids) - 1)
            # print(f"Input sentence: {tokenizer.decode(last_input_ids[random_index], skip_special_tokens=True)}")
            # print(f"Generated sentence: {last_decoded_preds[random_index]}")
            # print(f"Reference sentence: {last_decoded_labels[random_index][0]}")

        # if global_step % args.eval_every_steps == 0 or global_step == args.max_train_steps:
        #     eval_results, last_input_ids, last_decoded_preds, last_decoded_labels = evaluate_model(
        #         model=model,
        #         dataloader=eval_dataloader,
        #         target_tokenizer=target_tokenizer,
        #         device=args.device,
        #         max_seq_length=args.max_seq_length,
        #         generation_type=args.generation_type,
        #         beam_size=args.beam_size,
        #         max_eval_steps=args.max_eval_steps
        #     )
        #     # YOUR CODE ENDS HERE
        #     wandb.log(
        #         {
        #             "eval/bleu": eval_results["bleu"],
        #             "eval/generation_length": eval_results["generation_length"],
        #         },
        #         step=global_step,
        #     )
        #     logger.info("Generation example:")
        #     random_index = random.randint(0, len(last_input_ids) - 1)
        #     logger.info(f"Input sentence: {source_tokenizer.decode(last_input_ids[random_index], skip_special_tokens=True)}")
        #     logger.info(f"Generated sentence: {last_decoded_preds[random_index]}")
        #     logger.info(f"Reference sentence: {last_decoded_labels[random_index][0]}")


        #     logger.info("Saving model checkpoint to %s", args.output_dir)
        #     model.save_pretrained(args.output_dir)

        # if global_step >= args.max_train_steps:
        #     break

run.finish()  # stop wandb run


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


  0%|          | 0/1095 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

eval_err_num:1
easy pred: . join ( ',')
easy gold: SELECT count(*) FROM singer

eval_err_num:2
easy pred: @summary Total singers — — — — — — — — — — — — — — — — — — — — — — —
easy gold: SELECT count(*) FROM singer

eval_err_num:3
medium pred: name, country, age
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

eval_err_num:4
medium pred: name, country, age, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id, age_id
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

eval_err_num:5
medium pred: @name µµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµµ
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

eval_err_num:6
medium pred: @name FrenchSingers – – – – – – – – – – – – – – – – – – – – – – �
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France